# Collect Demonstration from Keyboard

Collect demonstration data for the given environment.
The task is to pick a mug and place it on the plate. The environment recognizes the success if the mug is on the plate, gthe ripper opened, and the end-effector positioned above the mug.

<img src="./media/teleop_v2.gif" width="480" height="360">

Use WASD for the xy plane, RF for the z-axis, QE for tilt, and ARROWs for the rest of rthe otations. 

SPACEBAR will change your gripper's state, and Z key will reset your environment with discarding the current episode data.

For overlayed images, 
- Top Right: Agent View 
- Bottom Right: Egocentric View
- Top Left: Left Side View
- Bottom Left: Top View

In [1]:
import sys
import random
import numpy as np
import os
import json
import itertools
import importlib
from PIL import Image
import mujoco_env.y_env2_remove_block_collect_dataset as collect_env
from lerobot.datasets.lerobot_dataset import LeRobotDataset
collect_env = importlib.reload(collect_env)
SimpleEnvRemoveBlockCollectDataset = collect_env.SimpleEnvRemoveBlockCollectDataset


/home/dragon/anaconda3/envs/mujoco_vla/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# If you want to randomize the object positions, set this to None
# If you fix the seed, the object positions will be the same every time
# SEED = 0 
SEED = 0 # 固定种子，确保方块总是位于盘子中心

REPO_NAME = 'remove_red_block_from_plate_so100_smolvla_mujoco'
NUM_DEMO = 20 # Number of demonstrations to collect
ROOT = "./demo_data_language" # The root directory to save the demonstrations

# ==================== 随机Block位置配置 ====================
ENABLE_RANDOM_BLOCK = True  # 禁用随机位置，始终固定在中心
PLATE_FLAT_RADIUS = 0.102   # 基于精确分析的平整区域半径（米）
BLOCK_SPAWN_MARGIN = 0.01  # 额外留出的安全裕度 (米)
Block_size = 0.015
Safe_dis = 0.015
effective_safe_radius = max(PLATE_FLAT_RADIUS - Block_size * np.sqrt(2) - BLOCK_SPAWN_MARGIN, 0.0)

print(f"🎯 配置信息:")
print(f"  随机Block位置: {'启用' if ENABLE_RANDOM_BLOCK else '禁用'}")
print(f"  数据集名称: {REPO_NAME}")
print(f"  Episode数量: {NUM_DEMO}")
print(f"  随机种子: {SEED}")
print(f"  Block生成安全裕度: {BLOCK_SPAWN_MARGIN:.3f} m")
if ENABLE_RANDOM_BLOCK:
    print(f"  Plate平整区域半径: {PLATE_FLAT_RADIUS:.3f}m")
    print(f"  采样安全半径(中心到块中心): {effective_safe_radius:.3f}m")


🎯 配置信息:
  随机Block位置: 启用
  数据集名称: remove_red_block_from_plate_so100_smolvla_mujoco
  Episode数量: 20
  随机种子: 0
  Block生成安全裕度: 0.010 m
  Plate平整区域半径: 0.102m
  采样安全半径(中心到块中心): 0.071m


In [3]:
# SAMPLING_METHOD = 'uniform'
SAMPLING_METHOD = 'latin_hypercube'

xml_path = './asset/scene_remove_block_so100.xml'
seed_it = itertools.count() if SEED is None else None
initial_seed = SEED if SEED is not None else next(seed_it)
# Define the environment with random block position support
PnPEnv = SimpleEnvRemoveBlockCollectDataset(
    xml_path, 
    seed=initial_seed, 
    state_type='joint_angle',
    random_block_position=ENABLE_RANDOM_BLOCK,
    plate_flat_radius=PLATE_FLAT_RADIUS,
    sampling_method=SAMPLING_METHOD,
    block_spawn_margin=BLOCK_SPAWN_MARGIN,
)



-----------------------------------------------------------------------------
name:[Tabletop] dt:[0.002] HZ:[500]
 n_qpos:[27] n_qvel:[24] n_qacc:[24] n_ctrl:[6]
 integrator:[RK4]

n_body:[18]
 [0/18] [world] mass:[0.00]kg
 [1/18] [front_object_table] mass:[1.00]kg
 [2/18] [camera] mass:[0.00]kg
 [3/18] [camera2] mass:[0.00]kg
 [4/18] [camera3] mass:[0.00]kg
 [5/18] [Base] mass:[0.56]kg
 [6/18] [Rotation_Pitch] mass:[0.12]kg
 [7/18] [Upper_Arm] mass:[0.16]kg
 [8/18] [Lower_Arm] mass:[0.15]kg
 [9/18] [Wrist_Pitch_Roll] mass:[0.07]kg
 [10/18] [Fixed_Jaw] mass:[0.09]kg
 [11/18] [camera_center] mass:[0.00]kg
 [12/18] [Moving_Jaw] mass:[0.02]kg
 [13/18] [body_obj_plate_11] mass:[0.00]kg
 [14/18] [object_plate_11] mass:[0.10]kg
 [15/18] [body_obj_mug_6] mass:[0.00]kg
 [16/18] [object_mug_6] mass:[0.08]kg
 [17/18] [body_obj_block_red] mass:[0.30]kg
body_total_mass:[2.66]kg

n_geom:[101]
geom_names:['floor', 'front_object_table', None, None, None, None, None, None, None, None, None, None, Non

In [4]:
# ==================== 简单位置测试 ====================
print("🧪 Block位置生成测试")
print("=" * 40)

if ENABLE_RANDOM_BLOCK:
    print(f"📍 采样方法: {SAMPLING_METHOD}")
    # 生成3个测试位置
    for i in range(3):
        PnPEnv.reset()
        block_pose, _, _ = PnPEnv.get_obj_pose()
        print(f"  位置 {i+1}: [{block_pose[0]:.3f}, {block_pose[1]:.3f}, {block_pose[2]:.3f}]")
    print("✅ 随机位置生成正常")
else:
    PnPEnv.reset()
    block_pose, _, _ = PnPEnv.get_obj_pose()
    print(f"📍 固定位置: [{block_pose[0]:.3f}, {block_pose[1]:.3f}, {block_pose[2]:.3f}]")

print("=" * 40)


🧪 Block位置生成测试
📍 采样方法: latin_hypercube
ik_err:[0.0163] is higher than ik_err_th:[0.0100].
You may want to increase max_ik_tick:[1000]
随机生成red block位置: [0.370, -0.194, 0.847]
Task instruction: Remove the red block from the plate
DONE INITIALIZATION
  位置 1: [0.370, -0.194, 0.824]
ik_err:[0.0163] is higher than ik_err_th:[0.0100].
You may want to increase max_ik_tick:[1000]
随机生成red block位置: [0.347, -0.209, 0.847]
Task instruction: Remove the red block from the plate
DONE INITIALIZATION
  位置 2: [0.349, -0.208, 0.823]
ik_err:[0.0163] is higher than ik_err_th:[0.0100].
You may want to increase max_ik_tick:[1000]
随机生成red block位置: [0.266, -0.154, 0.847]
Task instruction: Remove the red block from the plate
DONE INITIALIZATION
  位置 3: [0.266, -0.154, 0.823]
✅ 随机位置生成正常


## Define Dataset Fatures and Create your dataset!
The dataset is contained as follows:
```
fps = 20,
features={
    "observation.image": {
        "dtype": "image",
        "shape": (256, 256, 3),
        "names": ["height", "width", "channels"],
    },
    "observation.wrist_image": {
        "dtype": "image",
        "shape": (256, 256, 3),
        "names": ["height", "width", "channel"],
    },
    "observation.state": {
        "dtype": "float32",
        "shape": (6,),
        "names": ["state"], # x, y, z, roll, pitch, yaw
    },
    "action": {
        "dtype": "float32",
        "shape": (7,),
        "names": ["action"], # 6 joint angles and 1 gripper
    },
    "obj_init": {
        "dtype": "float32",
        "shape": (6,),
        "names": ["obj_init"], # just the initial position of the object. Not used in training.
    },
},
```


This will make the dataset on './demo_data' folder, which will look like this,

```
.
├── data
│   ├── chunk-000
│   │   ├── episode_000000.parquet
│   │   └── ...
├── meta
│   ├── episodes.jsonl
│   ├── info.json
│   ├── stats.json
│   └── tasks.jsonl
└── 
```


In [5]:
# ==================== 快速配置选项 ====================

print("🚀 Block位置配置选项")
print("=" * 50)
print()
print("📋 参数说明:")
print("  ENABLE_RANDOM_BLOCK: True=随机位置, False=固定位置")
print("  SAMPLING_METHOD: 'uniform'=均匀随机, 'latin_hypercube'=拉丁超立方")
print("  NUM_DEMO: 要收集的episode数量")
print()
print("🎯 推荐配置:")
print("1. 固定位置: ENABLE_RANDOM_BLOCK = False")
print("2. 均匀随机: ENABLE_RANDOM_BLOCK = True, SAMPLING_METHOD = 'uniform'")
print("3. 拉丁超立方: ENABLE_RANDOM_BLOCK = True, SAMPLING_METHOD = 'latin_hypercube'")
print("=" * 50)


🚀 Block位置配置选项

📋 参数说明:
  ENABLE_RANDOM_BLOCK: True=随机位置, False=固定位置
  SAMPLING_METHOD: 'uniform'=均匀随机, 'latin_hypercube'=拉丁超立方
  NUM_DEMO: 要收集的episode数量

🎯 推荐配置:
1. 固定位置: ENABLE_RANDOM_BLOCK = False
2. 均匀随机: ENABLE_RANDOM_BLOCK = True, SAMPLING_METHOD = 'uniform'
3. 拉丁超立方: ENABLE_RANDOM_BLOCK = True, SAMPLING_METHOD = 'latin_hypercube'


In [6]:
create_new = True
if os.path.exists(ROOT):
    print(f"Directory {ROOT} already exists.")
    ans = input("Do you want to delete it? (y/n) ")
    if ans == 'y':
        import shutil
        shutil.rmtree(ROOT)
    else:
        create_new = False


if create_new:
    dataset = LeRobotDataset.create(
                repo_id=REPO_NAME,
                root = ROOT, 
                robot_type="omy",
                fps=20, # 20 frames per second
                features={
                    "observation.image": {
                        "dtype": "image",
                        "shape": (256, 256, 3),
                        "names": ["height", "width", "channels"],
                    },
                    "observation.wrist_image": {
                        "dtype": "image",
                        "shape": (256, 256, 3),
                        "names": ["height", "width", "channel"],
                    },
                    "observation.state": {
                        "dtype": "float32",
                        "shape": (6,),
                        "names": ["state"], # x, y, z, roll, pitch, yaw
                    },
                    "observation.block_pose": {
                        "dtype": "float32",
                        "shape": (7,),
                        "names": ["block_pose"], # [x, y, z, qw, qx, qy, qz]
                    },
                    "action": {
                        "dtype": "float32",
                        "shape": (7,),
                        "names": ["action"], # 6 joint angles and 1 gripper
                    },
                    "obj_init": {
                        "dtype": "float32",
                        "shape": (9,),
                        "names": ["obj_init"], # just the initial position of the object. Not used in training.
                    },
                },
                image_writer_threads=10,
                image_writer_processes=5,
        )
else:
    print("Load from previous dataset")
    dataset = LeRobotDataset(REPO_NAME, root=ROOT)

Directory ./demo_data_language already exists.


## Keyboard Control
You can teleop your robot with keyboard and collect dataset
```
---------     -----------------------
   w       ->        backward
s  a  d        left   forward   right
---------      -----------------------
In x, y plane

---------
R: Moving Up
F: Moving Down
---------
In z axis

---------
Q: Tilt left
E: Tilt right
UP: Look Upward
Down: Look Donward
Right: Turn right
Left: Turn left
---------
For rotation

---------
SPACEBAR: Toggle Gripper
--------

---------
z: reset
--------
```
Reseting your environment will remove the cache data of the current demonstration and restart collection.

### Now let's teleop our robot and collect data!

**To receive the success signal, you have to release the gripper and move upwards above the mug!**

In [7]:
block_pose_records = []
block_pose_file = os.path.join(ROOT, REPO_NAME, "block_pose_log.json")
action = np.zeros(7)
episode_id = 0
record_flag = False # Start recording when the robot starts moving
while PnPEnv.env.is_viewer_alive() and episode_id < NUM_DEMO:
    PnPEnv.step_env()
    if PnPEnv.env.loop_every(HZ=20):
        # check if the episode is done
        done = PnPEnv.check_success()
        if done:
            # 记录操作耗时并保存episode数据
            episode_duration = PnPEnv.get_last_episode_duration()
            if episode_duration == 0.0:
                episode_duration = PnPEnv.get_episode_elapsed_time()
            dataset.save_episode()
            record = {
                "episode": int(episode_id),
                "block_pose": [float(v) for v in PnPEnv.get_block_stable_pose()],
                "duration_sec": float(episode_duration)
            }
            block_pose_records.append(record)
            print(f"Episode {episode_id} 完成，耗时 {episode_duration:.2f} 秒")
            # 实时保存日志文件，避免中断导致数据丢失
            os.makedirs(os.path.join(ROOT, REPO_NAME), exist_ok=True)
            with open(block_pose_file, 'w', encoding='utf-8') as f:
                json.dump(block_pose_records, f, ensure_ascii=False, indent=2)
            episode_id += 1
            record_flag = False
            # 每次成功后根据seed策略重新初始化
            if seed_it is not None:
                current_seed = next(seed_it)
                PnPEnv.reset(seed=current_seed)
            else:
                PnPEnv.reset()
            continue
        # Teleoperate the robot and get delta end-effector pose with gripper
        action, reset  = PnPEnv.teleop_robot()
        if not record_flag and np.any(action):
            record_flag = True
            print("Start recording")
        if reset:
            if seed_it is not None:
                current_seed = next(seed_it)
                PnPEnv.reset(seed=current_seed)
            else:
                PnPEnv.reset()
            # Reset the environment and clear the episode buffer
            # This can be done by pressing 'z' key
            dataset.clear_episode_buffer()
            record_flag = False
            continue
        # Step the environment
        # Get the end-effector pose and images
        agent_image, wrist_image = PnPEnv.grab_image()
        # # resize to 256x256
        agent_image = Image.fromarray(agent_image)
        wrist_image = Image.fromarray(wrist_image)
        agent_image = agent_image.resize((256, 256))
        wrist_image = wrist_image.resize((256, 256))
        agent_image = np.array(agent_image)
        wrist_image = np.array(wrist_image)
        joint_q = PnPEnv.step(action)
        action = PnPEnv.get_action_record() # 6 joint angles and 1 gripper
        action = action.astype(np.float32)
        if record_flag:
            # Add the frame to the dataset
            dataset.add_frame( {
                    "observation.image": agent_image,
                    "observation.wrist_image": wrist_image,
                    "observation.state": joint_q[:6],
                    "observation.block_pose": PnPEnv.get_block_stable_pose(),
                    "action": action,
                    "obj_init": PnPEnv.obj_init_pose,
                    # "task": PnPEnv.instruction,
                }, task = PnPEnv.instruction
            )
        PnPEnv.render(teleop=True, idx=episode_id)
if block_pose_records:
    os.makedirs(os.path.join(ROOT, REPO_NAME), exist_ok=True)
    with open(block_pose_file, 'w', encoding='utf-8') as f:
        json.dump(block_pose_records, f, ensure_ascii=False, indent=2)
    print(f"✅ 已保存block稳定位姿日志: {block_pose_file}")
else:
    print("⚠️ 未收集到完整episode，未生成block位姿日志。")



Start recording


Creating parquet from Arrow format: 100%|██████████| 3/3 [00:00<00:00, 140.01ba/s]


Episode 0 完成，耗时 17.77 秒
ik_err:[0.0163] is higher than ik_err_th:[0.0100].
You may want to increase max_ik_tick:[1000]
随机生成red block位置: [0.302, -0.224, 0.847]
Task instruction: Remove the red block from the plate
DONE INITIALIZATION
Start recording


Creating parquet from Arrow format: 100%|██████████| 3/3 [00:00<00:00, 142.63ba/s]


Episode 1 完成，耗时 11.04 秒
ik_err:[0.0163] is higher than ik_err_th:[0.0100].
You may want to increase max_ik_tick:[1000]
随机生成red block位置: [0.258, -0.193, 0.847]
Task instruction: Remove the red block from the plate
DONE INITIALIZATION
Start recording


...

Episode 3 完成，耗时 7.24 秒
ik_err:[0.0163] is higher than ik_err_th:[0.0100].
You may want to increase max_ik_tick:[1000]
随机生成red block位置: [0.282, -0.264, 0.847]
Task instruction: Remove the red block from the plate
DONE INITIALIZATION
Start recording


Creating parquet from Arrow format: 100%|██████████| 2/2 [00:00<00:00, 106.55ba/s]


Episode 4 完成，耗时 9.41 秒
ik_err:[0.0163] is higher than ik_err_th:[0.0100].
You may want to increase max_ik_tick:[1000]
随机生成red block位置: [0.294, -0.241, 0.847]
Task instruction: Remove the red block from the plate
DONE INITIALIZATION
Start recording


Creating parquet from Arrow format: 100%|██████████| 2/2 [00:00<00:00, 117.47ba/s]


Episode 5 完成，耗时 7.98 秒
ik_err:[0.0163] is higher than ik_err_th:[0.0100].
You may want to increase max_ik_tick:[1000]
随机生成red block位置: [0.331, -0.208, 0.847]
Task instruction: Remove the red block from the plate
DONE INITIALIZATION
Start recording
✅ 已保存block稳定位姿日志: ./demo_data_language/remove_red_block_from_plate_so100_smolvla_mujoco/block_pose_log.json


In [8]:
# ==================== 使用总结 ====================

print("🎯 随机Block位置数据收集 - 使用总结")
print("=" * 60)
print()
print("📋 数据收集流程：")
print("1️⃣ 在Cell 3中配置参数：")
print("   - ENABLE_RANDOM_BLOCK: 启用/禁用随机位置")
print("   - NUM_DEMO: 要收集的episode数量") 
print("   - SEED: 随机种子设置")
print("2️⃣ 运行Cell 3更新配置")
print("3️⃣ 运行Cell 4创建环境")
print("4️⃣ 运行Cell 5测试随机位置生成")
print("5️⃣ 运行Cell 6创建数据集")
print("6️⃣ 运行Cell 12开始数据收集")
print()
print("✨ 新功能特点：")
print("   - 🎯 基于精确几何分析的随机位置生成")
print("   - 🔒 确保block四个角都在plate平整区域内")
print("   - 🎮 保持原有的键盘控制体验")
print("   - 🔄 支持固定位置和随机位置切换")
print("   - 📊 自动显示位置分布统计")
print()
print("🎯 推荐数据收集策略：")
strategies = [
    "策略1: 先收集固定位置基线数据 (ENABLE_RANDOM_BLOCK=False)",
    "策略2: 再收集随机位置数据增强泛化性 (ENABLE_RANDOM_BLOCK=True)",
    "策略3: 使用不同随机种子收集多组数据",
    "策略4: 根据训练效果调整NUM_DEMO数量"
]
for strategy in strategies:
    print(f"   {strategy}")

print()
print("🔧 参数调优建议：")
print("   - 初学者: ENABLE_RANDOM_BLOCK=False, NUM_DEMO=20")
print("   - 进阶用户: ENABLE_RANDOM_BLOCK=True, NUM_DEMO=50")
print("   - 研究用途: 收集多组不同配置的数据集")
print()
print("💡 提示：每次修改配置后，重新运行Cell 3-5即可！")
print("=" * 60)


🎯 随机Block位置数据收集 - 使用总结

📋 数据收集流程：
1️⃣ 在Cell 3中配置参数：
   - ENABLE_RANDOM_BLOCK: 启用/禁用随机位置
   - NUM_DEMO: 要收集的episode数量
   - SEED: 随机种子设置
2️⃣ 运行Cell 3更新配置
3️⃣ 运行Cell 4创建环境
4️⃣ 运行Cell 5测试随机位置生成
5️⃣ 运行Cell 6创建数据集
6️⃣ 运行Cell 12开始数据收集

✨ 新功能特点：
   - 🎯 基于精确几何分析的随机位置生成
   - 🔒 确保block四个角都在plate平整区域内
   - 🎮 保持原有的键盘控制体验
   - 🔄 支持固定位置和随机位置切换
   - 📊 自动显示位置分布统计

🎯 推荐数据收集策略：
   策略1: 先收集固定位置基线数据 (ENABLE_RANDOM_BLOCK=False)
   策略2: 再收集随机位置数据增强泛化性 (ENABLE_RANDOM_BLOCK=True)
   策略3: 使用不同随机种子收集多组数据
   策略4: 根据训练效果调整NUM_DEMO数量

🔧 参数调优建议：
   - 初学者: ENABLE_RANDOM_BLOCK=False, NUM_DEMO=20
   - 进阶用户: ENABLE_RANDOM_BLOCK=True, NUM_DEMO=50
   - 研究用途: 收集多组不同配置的数据集

💡 提示：每次修改配置后，重新运行Cell 3-5即可！


In [9]:
PnPEnv.env.close_viewer()